### load csv [for 연봉]

In [67]:
import numpy as np
import pandas as pd
# 선수, 팀 데이터
PATH_P = "C:/Users/home/DA(inflearn)/individual_pitcher/2020빅콘테스트 문제데이터(데이터분석분야-퓨처스리그)/2020빅콘테스트 데이터분석분야-퓨쳐스리그_스포츠투아이_제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_선수_"
PATH_T = "C:/Users/home/DA(inflearn)/individual_pitcher/2020빅콘테스트 문제데이터(데이터분석분야-퓨처스리그)/2020빅콘테스트 데이터분석분야-퓨쳐스리그_스포츠투아이_제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀_"

doc_p = None
doc_t = None
for i in [2016, 2017, 2018, 2019, 2020]:
    doc_i = pd.read_csv(PATH_P+str(i)+".csv", encoding='CP949')
    doc_p = pd.concat([doc_p, doc_i], axis=0)
    doc_j = pd.read_csv(PATH_T+str(i)+".csv", encoding='CP949')
    doc_t = pd.concat([doc_t, doc_j], axis=0)

# 시즌에 따라 팀명이 바뀌는 지 확인
doc_t.iloc[:,0].value_counts() #안 바뀜

OB    5
LT    5
NC    5
WO    5
SK    5
SS    5
HT    5
LG    5
HH    5
KT    5
Name: T_ID, dtype: int64

### 데이터 가공

In [68]:
# 팀+선수 & 컬럼 순서 교체
doc_t = pd.read_csv(PATH_T+"2020"+".csv", encoding='CP949')
p_df = pd.merge(doc_p, doc_t, on="T_ID", how="left")
p_df = p_df[["GYEAR","PCODE","NAME","T_ID","T_NM","POSITION","AGE_VA","MONEY"]]
p_df

,GYEAR,PCODE,NAME,T_ID,T_NM,POSITION,AGE_VA,MONEY
0,2016,60100,백창수,LG,LG,내,28,4000만원
1,2016,60105,배민관,LG,LG,투,25,2700만원
2,2016,60146,이승현,LG,LG,투,25,3700만원
3,2016,60164,유경국,LG,LG,투,25,2700만원
4,2016,60181,김지용,LG,LG,투,28,4000만원
...,...,...,...,...,...,...,...,...
3266,2020,79705,김회성,HH,한화,내,35,6500만원
3267,2020,79764,장민재,HH,한화,투,30,11000만원
3268,2020,79825,여건욱,LG,LG,투,34,7000만원
3269,2020,79847,김태훈,SK,SK,투,30,24000만원


In [69]:
# 컬럼명 변경
p_df.columns = ["시즌", "선수코드", "선수명", "팀코드", "팀명", "포지션", "나이", "연봉"]
p_df = p_df.set_index(pd.Index(range(0,p_df.shape[0])))
p_df

,시즌,선수코드,선수명,팀코드,팀명,포지션,나이,연봉
0,2016,60100,백창수,LG,LG,내,28,4000만원
1,2016,60105,배민관,LG,LG,투,25,2700만원
2,2016,60146,이승현,LG,LG,투,25,3700만원
3,2016,60164,유경국,LG,LG,투,25,2700만원
4,2016,60181,김지용,LG,LG,투,28,4000만원
...,...,...,...,...,...,...,...,...
3266,2020,79705,김회성,HH,한화,내,35,6500만원
3267,2020,79764,장민재,HH,한화,투,30,11000만원
3268,2020,79825,여건욱,LG,LG,투,34,7000만원
3269,2020,79847,김태훈,SK,SK,투,30,24000만원


In [70]:
# 결측치 확인
p_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3271 entries, 0 to 3270
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시즌      3271 non-null   int64 
 1   선수코드    3271 non-null   int64 
 2   선수명     3271 non-null   object
 3   팀코드     3271 non-null   object
 4   팀명      3271 non-null   object
 5   포지션     3271 non-null   object
 6   나이      3271 non-null   int64 
 7   연봉      3270 non-null   object
dtypes: int64(3), object(5)
memory usage: 204.6+ KB


In [71]:
# 결측치 정제
p_df = p_df.fillna({"연봉":"0만원"})
print(p_df[["연봉"]].isnull().sum())

연봉    0
dtype: int64


In [72]:
# 연봉 단위 확인
unit = pd.Series(index=range(p_df.shape[0]))
for i, row in p_df.iterrows():
    unit[i] = p_df.at[i,"연봉"][-2:]
unit.value_counts()

C:\Users\home\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



만원    3091
달러     179
0엔       1
dtype: int64

In [73]:
# 단위 통일 + 환율 계산
for i, row in p_df.iterrows():
    money = int(p_df.at[i,"연봉"][:-2])
    if(p_df.at[i,"연봉"][-2:]=="달러"):
        money *= 0.11
        p_df.at[i,"연봉"] = money
    elif(p_df.at[i,"연봉"][-2:]=="0엔"):
        money = int(p_df.at[i,"연봉"][:-1])
        money *= 0.0011
        p_df.at[i,"연봉"] = money
    else:
        p_df.at[i,"연봉"] = money
p_df[["연봉"]]

,연봉
0,4000
1,2700
2,3700
3,2700
4,4000
...,...
3266,6500
3267,11000
3268,7000
3269,24000


In [74]:
# 확인
p_df.iloc[54]

시즌        2016
선수코드     61240
선수명        니퍼트
팀코드         OB
팀명          두산
포지션          투
나이          35
연봉      132000
Name: 54, dtype: object

In [75]:
1200000*0.11

132000.0

In [76]:
# 확인2
20000000*0.0011

22000.0

In [77]:
p_df.iloc[367]

시즌       2016
선수코드    66748
선수명     마에스트리
팀코드        HH
팀명         한화
포지션         투
나이         31
연봉      22000
Name: 367, dtype: object

In [78]:
# 데이터 타입 확인
p_df[["연봉"]].dtypes

연봉    object
dtype: object

In [79]:
# 데이터 타입 변경(obj -> float)
p_df[["연봉"]] = p_df[["연봉"]].astype(str).astype("float64")
p_df[["연봉"]]

,연봉
0,4000.0
1,2700.0
2,3700.0
3,2700.0
4,4000.0
...,...
3266,6500.0
3267,11000.0
3268,7000.0
3269,24000.0


In [108]:
# 팀별 연봉 계산(sum)
t_df = p_df.groupby(by=["시즌","팀코드","팀명"])
t_df_sum = t_df.sum().sort_values(by=["시즌","연봉"])[["연봉"]]
t_df_sum = t_df_sum.reset_index()
t_df_sum

,시즌,팀코드,팀명,연봉
0,2016,WO,키움,577200.0
1,2016,KT,KT,726300.0
2,2016,NC,NC,872200.0
3,2016,LG,LG,925700.0
4,2016,SK,SK,955500.0
5,2016,OB,두산,1005900.0
6,2016,HT,KIA,1058700.0
7,2016,LT,롯데,1088400.0
8,2016,SS,삼성,1116400.0
9,2016,HH,한화,1454500.0


In [82]:
# 팀명 serise
tim_labels = t_df_sum.iloc[0:10,1]
tim_labels

0    WO
1    KT
2    NC
3    LG
4    SK
5    OB
6    HT
7    LT
8    SS
9    HH
Name: 팀코드, dtype: object

### 시각화

In [83]:
# 시즌/팀 별 연봉 그래프 (Bar)
import plotly.express as px
fig = px.bar(t_df_sum, x="연봉", y="팀명", barmode="group",
             facet_row="시즌", orientation='h',
             category_orders={"시즌": ["2016", "2017", "2018", "2019", "2020"]},
             title="<b>시즌/팀</b> 별 연봉 그래프", width=1000, height=1000,
             color="팀명")
fig.show()

In [84]:
# Line 그래프를 위한 데이터 가공
t_df = p_df.groupby(by=["팀코드","팀명","시즌"])
t_df_sum_line = t_df.sum()[["연봉"]]
t_df_sum_line = t_df_sum.reset_index()
t_df_sum_line

,index,시즌,팀코드,팀명,연봉
0,0,2016,WO,키움,577200.0
1,1,2016,KT,KT,726300.0
2,2,2016,NC,NC,872200.0
3,3,2016,LG,LG,925700.0
4,4,2016,SK,SK,955500.0
5,5,2016,OB,두산,1005900.0
6,6,2016,HT,KIA,1058700.0
7,7,2016,LT,롯데,1088400.0
8,8,2016,SS,삼성,1116400.0
9,9,2016,HH,한화,1454500.0


In [86]:
# Line 그래프를 위한 데이터 가공2
tims = [0]*10
for i in range(10):
    tims[i] = t_df_sum_line[t_df_sum_line["팀코드"]==tim_labels[i]]
tims[1]

,index,시즌,팀코드,팀명,연봉
1,1,2016,KT,KT,726300.0
10,10,2017,KT,KT,639600.0
21,21,2018,KT,KT,807300.0
30,30,2019,KT,KT,706700.0
40,40,2020,KT,KT,750700.0


In [88]:
# 시즌/팀 별 연봉 그래프<sum> (Line)
import plotly.graph_objects as go
fig = go.Figure()
for i in range(len(tims)):
    fig.add_trace(go.Scatter(x=tims[i].iloc[:,1], y=tims[i].iloc[:,4],
                             mode='lines+markers',
                             name=tims[i].iloc[0,3]))
fig.update_layout(
    {
        "title": {
            "text": "<b>시즌/팀</b> 별 연봉 그래프 <b><sum></b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "showlegend": True,
        "xaxis": {
            "title": "시즌",
            "showticklabels":True,
            "dtick":1 
            
        },
        "yaxis": {
            "title": "수치"
        },

    }
)

fig.show()

### load csv [for 승률]

In [90]:
# 팀투수 데이터
PATH_TP = "C:/Users/home/DA(inflearn)/individual_pitcher/2020빅콘테스트 문제데이터(데이터분석분야-퓨처스리그)/2020빅콘테스트 데이터분석분야-퓨쳐스리그_스포츠투아이_제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀투수_"
doc_tp = None
for i in [2016, 2017, 2018, 2019, 2020]:
    doc_i = pd.read_csv(PATH_TP+str(i)+".csv", encoding='CP949')
    doc_tp = pd.concat([doc_tp, doc_i], axis=0)
tp_df = doc_tp[["G_ID","T_ID","WLS"]]
tp_df = tp_df.set_index(pd.Index(range(0,tp_df.shape[0])))

# G_ID를 이용하여 년도 데이터 획득
for i, row in tp_df.iterrows():
    tp_df.at[i,"G_ID"] = tp_df.at[i,"G_ID"][:4]
    
tp_df.columns = ["시즌","팀코드","승패"]
tp_df = tp_df.set_index(pd.Index(range(0,tp_df.shape[0])))
tp_df

,시즌,팀코드,승패
0,2016,LG,W
1,2016,HH,L
2,2016,NC,W
3,2016,HT,L
4,2016,SK,L
...,...,...,...
6395,2020,LT,W
6396,2020,HT,L
6397,2020,OB,W
6398,2020,SK,W


### 데이터 가공

In [91]:
# 승패 데이터를 이진 데이터로 (0/1)
for i, row in tp_df.iterrows():
    if(tp_df.at[i,"승패"]=="W"):
        tp_df.at[i,"승패"] = 1
    else:
        tp_df.at[i,"승패"] = 0
tp_df

,시즌,팀코드,승패
0,2016,LG,1
1,2016,HH,0
2,2016,NC,1
3,2016,HT,0
4,2016,SK,0
...,...,...,...
6395,2020,LT,1
6396,2020,HT,0
6397,2020,OB,1
6398,2020,SK,1


In [92]:
# 시즌/팀별로 승률 계산
grouped = tp_df.groupby(by=["시즌","팀코드"])
win_rate = grouped.agg(np.sum)/grouped.agg(np.size)
win_rate = win_rate.reset_index()
win_rate = win_rate.sort_values(by=["팀코드","시즌"])
win_rate

,시즌,팀코드,승패
0,2016,HH,0.458333
10,2017,HH,0.423611
20,2018,HH,0.534722
30,2019,HH,0.402778
40,2020,HH,0.261538
1,2016,HT,0.486111
11,2017,HT,0.604167
21,2018,HT,0.486111
31,2019,HT,0.430556
41,2020,HT,0.532258


### 시각화

In [93]:
# Line 그래프를 위한 데이터 가공
tims_bar = [0]*10 #원래 바그래프를 위한 데이터였으나, 좀 더 직관적인 시각화를 위해 라인 그래프로 변경
for i in range(10):
    tims_bar[i] = win_rate[win_rate["팀코드"]==tim_labels[i]]
tims_bar

[      시즌 팀코드        승패
 9   2016  WO  0.534722
 19  2017  WO  0.479167
 29  2018  WO  0.520833
 39  2019  WO  0.597222
 49  2020  WO  0.575758,
       시즌 팀코드        승패
 2   2016  KT  0.368056
 12  2017  KT  0.347222
 22  2018  KT  0.409722
 32  2019  KT  0.493056
 42  2020  KT  0.484375,
       시즌 팀코드        승패
 5   2016  NC  0.576389
 15  2017  NC  0.548611
 25  2018  NC  0.402778
 35  2019  NC  0.506944
 45  2020  NC  0.666667,
       시즌 팀코드        승패
 3   2016  LG  0.493056
 13  2017  LG  0.479167
 23  2018  LG  0.472222
 33  2019  LG  0.548611
 43  2020  LG  0.531250,
       시즌 팀코드        승패
 7   2016  SK  0.479167
 17  2017  SK  0.520833
 27  2018  SK  0.541667
 37  2019  SK  0.611111
 47  2020  SK  0.323077,
       시즌 팀코드        승패
 6   2016  OB  0.645833
 16  2017  OB  0.583333
 26  2018  OB  0.645833
 36  2019  OB  0.611111
 46  2020  OB  0.593750,
       시즌 팀코드        승패
 1   2016  HT  0.486111
 11  2017  HT  0.604167
 21  2018  HT  0.486111
 31  2019  HT  0.430556
 41  2020 

In [94]:
# 시즌/팀 별 승률 그래프 (Line)
import plotly.graph_objects as go
fig = go.Figure()
for i in range(len(tims)):
    fig.add_trace(
        go.Scatter(
        x=tims_bar[i].iloc[:,0], y=tims_bar[i].iloc[:,2], name=tims[i].iloc[0,3], mode='lines+markers'
        )
    )
fig.update_layout(
    {
        "title": {
            "text": "<b>시즌/팀</b> 별 승률 그래프",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "showlegend": True,
        "xaxis": {
            "title": "시즌",
            "showticklabels":True,
            "dtick":1 
            
        },
        "yaxis": {
            "title": "수치"
        },

    }
)

fig.show()

In [95]:
# 시즌/팀 별 승률 그래프 (Bar)
import plotly.graph_objects as go
fig = go.Figure()
for i in range(len(tims)):
    fig.add_trace(
        go.Bar(
        x=tims_bar[i].iloc[:,0], y=tims_bar[i].iloc[:,2], name=tims[i].iloc[0,3], text=tims_bar[i].iloc[:,2], 
            textposition='auto', texttemplate='%{y:.2f}'
        )
    )
fig.update_layout(
    {
        "title": {
            "text": "<b>시즌/팀</b> 별 승률 그래프",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "showlegend": True,
        "xaxis": {
            "title": "시즌",
            "showticklabels":True,
            "dtick":1 
            
        },
        "yaxis": {
            "title": "수치"
        },

    }
)

fig.show()

In [96]:
# Line 그래프<mean>를 위한 데이터 가공
t_df = p_df.groupby(by=["팀코드","팀명","시즌"])
t_df_mean_line = t_df.mean()[["연봉"]]
t_df_mean_line = t_df_mean_line.reset_index()
#t_df_sum_line = t_df_sum_line.set_index(["팀코드"])

tims = [0]*10
for i in range(10):
    tims[i] = t_df_mean_line[t_df_mean_line["팀코드"]==tim_labels[i]]
tims[1]



,팀코드,팀명,시즌,연봉
10,KT,KT,2016,11173.846154
11,KT,KT,2017,9546.268657
12,KT,KT,2018,12420.000000
13,KT,KT,2019,11042.187500
14,KT,KT,2020,11729.687500


In [97]:
# 시즌/팀 별 연봉 그래프<mean> (Line)
import plotly.graph_objects as go
fig = go.Figure()
for i in range(len(tims)):
    fig.add_trace(go.Scatter(x=tims[i].iloc[:,2], y=tims[i].iloc[:,3],
                             mode='lines+markers',
                             name=tims[i].iloc[0,1]))
fig.update_layout(
    {
        "title": {
            "text": "<b>시즌/팀</b> 별 연봉 그래프 <b><mean></b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "showlegend": True,
        "xaxis": {
            "title": "시즌",
            "showticklabels":True,
            "dtick":1 
            
        },
        "yaxis": {
            "title": "수치"
        },

    }
)

fig.show()

In [30]:
t_df_mean_line 

,팀코드,팀명,시즌,연봉
0,HH,한화,2016,19655.405405
1,HH,한화,2017,21365.714286
2,HH,한화,2018,16545.522388
3,HH,한화,2019,13901.515152
4,HH,한화,2020,12501.428571
5,HT,KIA,2016,15801.492537
6,HT,KIA,2017,20091.176471
7,HT,KIA,2018,21858.208955
8,HT,KIA,2019,16777.142857
9,HT,KIA,2020,15842.698413


In [31]:
t_df_sum_line

,index,시즌,팀코드,팀명,연봉
0,0,2016,WO,키움,577200.0
1,1,2016,KT,KT,726300.0
2,2,2016,NC,NC,872200.0
3,3,2016,LG,LG,925700.0
4,4,2016,SK,SK,955500.0
5,5,2016,OB,두산,1005900.0
6,6,2016,HT,KIA,1058700.0
7,7,2016,LT,롯데,1088400.0
8,8,2016,SS,삼성,1116400.0
9,9,2016,HH,한화,1454500.0


### load csv [for 상대승률](by dojin)

In [100]:
# 상대 승률 데이터(출처: 도진이형)
from Win_rate_by_season_EDA_3 import win_rate as wr
wr

,T_ID,VS_T_ID,WLS_2016,WLS_2017,WLS_2018,WLS_2019
0,HH,HT,0.43750,0.31250,0.5625,0.2500
1,HH,KT,0.46875,0.75000,0.6250,0.5000
2,HH,LG,0.43750,0.56250,0.5625,0.4375
3,HH,LT,0.50000,0.31250,0.6250,0.5000
4,HH,NC,0.46875,0.40625,0.5625,0.4375
...,...,...,...,...,...,...
85,WO,LT,0.56250,0.37500,0.4375,0.8125
86,WO,NC,0.50000,0.25000,0.5625,0.6250
87,WO,OB,0.40625,0.50000,0.5000,0.5625
88,WO,SK,0.37500,0.59375,0.5625,0.5000


### 데이터 가공

In [101]:
wr = wr.rename({"T_ID":"팀코드"},axis="columns")
wr


,팀코드,VS_T_ID,WLS_2016,WLS_2017,WLS_2018,WLS_2019
0,HH,HT,0.43750,0.31250,0.5625,0.2500
1,HH,KT,0.46875,0.75000,0.6250,0.5000
2,HH,LG,0.43750,0.56250,0.5625,0.4375
3,HH,LT,0.50000,0.31250,0.6250,0.5000
4,HH,NC,0.46875,0.40625,0.5625,0.4375
...,...,...,...,...,...,...
85,WO,LT,0.56250,0.37500,0.4375,0.8125
86,WO,NC,0.50000,0.25000,0.5625,0.6250
87,WO,OB,0.40625,0.50000,0.5000,0.5625
88,WO,SK,0.37500,0.59375,0.5625,0.5000


In [102]:
# 팀연봉 데이터(출처: 동준)
import pandas as pd
PATH_TS = "C:/Users/home/DA(inflearn)/individual_pitcher/팀연봉.csv"
ts = pd.read_csv(PATH_TS)

# 필요한 데이터만 추출해서 상대승률 데이터에 합치기
re_ts = None
for i in tim_labels:
    sub = ts[ts['팀코드']==i].set_index('시즌').T.drop(["Unnamed: 0","팀코드","팀명","총합연봉"])
    sub["팀코드"] = i
    re_ts = pd.concat([re_ts, sub])
wr = pd.merge(wr, re_ts, on="팀코드", how="left")
wr = pd.merge(wr, re_ts, left_on="VS_T_ID", right_on="팀코드", how="left")
wr # _x: 자기팀 데이터, _y: 상대팀 데이터        

,팀코드_x,VS_T_ID,WLS_2016,WLS_2017,WLS_2018,WLS_2019,2016_x,2017_x,2018_x,2019_x,2020_x,2016_y,2017_y,2018_y,2019_y,2020_y,팀코드_y
0,HH,HT,0.43750,0.31250,0.5625,0.2500,19655.4,21365.7,16545.5,13901.5,12501.4,15801.5,20091.2,21858.2,16777.1,15842.7,HT
1,HH,KT,0.46875,0.75000,0.6250,0.5000,19655.4,21365.7,16545.5,13901.5,12501.4,11173.8,9546.27,12420,11042.2,11729.7,KT
2,HH,LG,0.43750,0.56250,0.5625,0.4375,19655.4,21365.7,16545.5,13901.5,12501.4,14025.8,15680,18098.4,15833.3,18178.7,LG
3,HH,LT,0.50000,0.31250,0.6250,0.5000,19655.4,21365.7,16545.5,13901.5,12501.4,15548.6,18554.5,20712.5,21316.6,16682.8,LT
4,HH,NC,0.46875,0.40625,0.5625,0.4375,19655.4,21365.7,16545.5,13901.5,12501.4,13017.9,14783.3,10625,17454.8,18982.8,NC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,WO,LT,0.56250,0.37500,0.4375,0.8125,9018.75,12907.6,15369.5,13222,14760,15548.6,18554.5,20712.5,21316.6,16682.8,LT
86,WO,NC,0.50000,0.25000,0.5625,0.6250,9018.75,12907.6,15369.5,13222,14760,13017.9,14783.3,10625,17454.8,18982.8,NC
87,WO,OB,0.40625,0.50000,0.5000,0.5625,9018.75,12907.6,15369.5,13222,14760,15013.4,18125,16895.6,17853,15476.2,OB
88,WO,SK,0.37500,0.59375,0.5625,0.5000,9018.75,12907.6,15369.5,13222,14760,14051.5,16798.4,17692.3,18597,15093.5,SK


In [103]:
# 연봉데이터를 이진데이터로: 높은 쪽이 1, 낮은 쪽 0
def compare(col1, col2):
    for i in range(len(col1)):
        if col1[i]>col2[i]:
            col1[i] = 1
            col2[i] = 0
        else:
            col1[i] = 0
            col2[i] = 1
for i in [2016,2017,2018,2019,2020]:
    compare(wr[str(i)+"_x"],wr[str(i)+"_y"])
wr

C:\Users\home\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\home\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\home\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\home\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning:


A value

,팀코드_x,VS_T_ID,WLS_2016,WLS_2017,WLS_2018,WLS_2019,2016_x,2017_x,2018_x,2019_x,2020_x,2016_y,2017_y,2018_y,2019_y,2020_y,팀코드_y
0,HH,HT,0.43750,0.31250,0.5625,0.2500,1,1,0,0,0,0,0,1,1,1,HT
1,HH,KT,0.46875,0.75000,0.6250,0.5000,1,1,1,1,1,0,0,0,0,0,KT
2,HH,LG,0.43750,0.56250,0.5625,0.4375,1,1,0,0,0,0,0,1,1,1,LG
3,HH,LT,0.50000,0.31250,0.6250,0.5000,1,1,0,0,0,0,0,1,1,1,LT
4,HH,NC,0.46875,0.40625,0.5625,0.4375,1,1,1,0,0,0,0,0,1,1,NC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,WO,LT,0.56250,0.37500,0.4375,0.8125,0,0,0,0,0,1,1,1,1,1,LT
86,WO,NC,0.50000,0.25000,0.5625,0.6250,0,0,1,0,0,1,1,0,1,1,NC
87,WO,OB,0.40625,0.50000,0.5000,0.5625,0,0,0,0,0,1,1,1,1,1,OB
88,WO,SK,0.37500,0.59375,0.5625,0.5000,0,0,0,0,0,1,1,1,1,1,SK


In [104]:
# 승률 데이터를 이진데이터로: 반올림
wr = round(wr)
wr

,팀코드_x,VS_T_ID,WLS_2016,WLS_2017,WLS_2018,WLS_2019,2016_x,2017_x,2018_x,2019_x,2020_x,2016_y,2017_y,2018_y,2019_y,2020_y,팀코드_y
0,HH,HT,0.0,0.0,1.0,0.0,1,1,0,0,0,0,0,1,1,1,HT
1,HH,KT,0.0,1.0,1.0,0.0,1,1,1,1,1,0,0,0,0,0,KT
2,HH,LG,0.0,1.0,1.0,0.0,1,1,0,0,0,0,0,1,1,1,LG
3,HH,LT,0.0,0.0,1.0,0.0,1,1,0,0,0,0,0,1,1,1,LT
4,HH,NC,0.0,0.0,1.0,0.0,1,1,1,0,0,0,0,0,1,1,NC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,WO,LT,1.0,0.0,0.0,1.0,0,0,0,0,0,1,1,1,1,1,LT
86,WO,NC,0.0,0.0,1.0,1.0,0,0,1,0,0,1,1,0,1,1,NC
87,WO,OB,0.0,0.0,0.0,1.0,0,0,0,0,0,1,1,1,1,1,OB
88,WO,SK,0.0,1.0,1.0,0.0,0,0,0,0,0,1,1,1,1,1,SK


In [105]:
# 빈도수 데이터 프레임
wlhl = pd.DataFrame([[0,0],[0,0]], columns=["높은 연봉","낮은 연봉"], index=["높은 승률","낮은 승률"])
wlhl

,높은 연봉,낮은 연봉
높은 승률,0,0
낮은 승률,0,0


In [106]:
# 데이터 프레임 채우기
def calculate(row,wlhl):
    for i in range(2,6):
        if row[i]==1 and row[4+i]==1:
            wlhl.at["높은 승률","높은 연봉"] += 1
        elif row[i]==1 and row[4+i]==0:
            wlhl.at["높은 승률","낮은 연봉"] += 1
        elif row[i]==0 and row[4+i]==1:
            wlhl.at["낮은 승률","높은 연봉"] += 1
        elif row[i]==0 and row[4+i]==0:
            wlhl.at["낮은 승률","낮은 연봉"] += 1
    return wlhl
for i, row in wr.iterrows():
    wlhl = calculate(row,wlhl)
wlhl

,높은 연봉,낮은 연봉
높은 승률,87,65
낮은 승률,93,115


### 데이터 분석(Chi-square)

In [53]:
# 카이제곱 독립성 검정
from scipy.stats import chi2_contingency

chi2_contingency(wlhl) #p-value = 0.025... < 0.05 => 두 변인은 관련이 있다.

(5.021508097165992,
 0.02503435619792018,
 1,
 array([[ 76.,  76.],
        [104., 104.]]))

In [ ]:
# create csv
salary_t = t_df_sum
salary_t.columns = ["시즌","팀코드","팀명","총합연봉"]
salary_t

In [ ]:
salary_t = pd.merge(salary_t,t_df_mean_line,on=["시즌","팀코드","팀명"], how="left")
salary_t.columns = ["시즌","팀코드","팀명","총합연봉","평균연봉"]
salary_t

In [50]:
salary_t.to_csv("팀연봉.csv",mode="w")
salary.to_csv("선수연봉.csv",mode="w")

NameError: name 'salary_t' is not defined